In [1]:
import shapefile as sf
import numpy as np
import sklearn as sl


from itertools import cycle, compress
from sklearn import preprocessing
from sklearn import neighbors
from metric_learn import LMNN
from multiprocessing import Pool


inputfile = sf.Reader("/g/data1a/ge3/AEM_Model/3D_AEM_model_V3.shp")

print(inputfile.fields)

[('DeletionFlag', 'C', 1, 0), ['Id', 'N', 10, 0], ['unique_id', 'C', 50, 0], ['ref_0001', 'F', 13, 11], ['ref_0001vs', 'F', 13, 11], ['ref_0010', 'F', 13, 11], ['ref_0100', 'F', 13, 11], ['ref_0100vs', 'F', 13, 11], ['thick', 'F', 13, 11], ['wii_clip_1', 'F', 13, 11], ['ceno_clip_', 'F', 13, 11], ['X', 'F', 19, 11], ['Y', 'F', 19, 11], ['Z', 'F', 19, 11], ['class', 'N', 10, 0], ['age', 'N', 10, 0]]


In [2]:
inputrecord = inputfile.records()

labelledrecord= np.array([x for x in inputrecord if x[-2]>0 and x[1]!='Surface'])
unlabelledrecord= np.array([x for x in inputrecord if x[-2]==0 and x[1]!='Surface'])

In [93]:
def run(train,test):
    traininglabel = train[:,-2]
    trainingfeature = train[:,[*range(2,10),12,14]]

    testinglabel = test[:,-2]
    testingfeature = test[:,[*range(2,10),12,14]]    
    
    whitener = preprocessing.StandardScaler().fit(trainingfeature)

    trainingfeature_whitened = whitener.transform(trainingfeature)
    testingfeature_whitened = whitener.transform(testingfeature)
    
    lmnn = LMNN(k=5, learn_rate=1e-6)
    lmnn.fit(trainingfeature_whitened, traininglabel)
    
    trainingfeature_transformed = lmnn.transform(trainingfeature_whitened)
    testingfeature_transformed = lmnn.transform(testingfeature_whitened)

    kdtree = neighbors.KDTree(trainingfeature_transformed)
    
    prediction = kdtree.query(testingfeature_transformed,1)
    
    predictedlabel = np.array([traininglabel[x[0]] for x in prediction[1]])
    
    #return testinglabel-predictedlabel
    return float(sum(map(bool,testinglabel.astype(int)-predictedlabel.astype(int))))/len(testinglabel)
    
def knn(train, test):
    traininglabel = train[:,-2]
    trainingfeature = train[:,[*range(2,10),12,14]]

    testinglabel = test[:,-2]
    testingfeature = test[:,[*range(2,10),12,14]]  
    
    whitener = preprocessing.StandardScaler().fit(trainingfeature)

    trainingfeature_whitened = whitener.transform(trainingfeature)
    testingfeature_whitened = whitener.transform(testingfeature)    
    
    lmnn = LMNN(k=5, learn_rate=1e-6)
    lmnn.fit(trainingfeature_whitened, traininglabel)    
    
    trainingfeature_transformed = lmnn.transform(trainingfeature_whitened)
    testingfeature_transformed = lmnn.transform(testingfeature_whitened)    
    
    knn=neighbors.KNeighborsClassifier(n_neighbors=5)
    knn.fit(trainingfeature_transformed, traininglabel)
    predictedlabel = knn.predict(testingfeature_transformed)
    
    return float(sum(map(bool,testinglabel.astype(int)-predictedlabel.astype(int))))/len(testinglabel)
    
    
def cross_fold( fid):
    folds = sorted(list(set(labelledrecord[:,1])))
    
    train = np.array([x for x in labelledrecord if x[1]!=folds[fid]])
    test = np.array([x for x in labelledrecord if x[1]==folds[fid]])
    return knn(train, test)


In [94]:
error = Pool(10).map(cross_fold,range(51))



/g/data1a/ge3/yuhang/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype <U21 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/g/data1a/ge3/yuhang/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype <U21 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/g/data1a/ge3/yuhang/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype <U21 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/g/data1a/ge3/yuhang/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype <U21 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/g/data1a/ge3/yuhang/anaconda3/lib/python3.6/site-packages/sklearn/utils/validat

In [95]:
np.mean(error)

0.19511715017390968

In [96]:
error

[0.1,
 0.06666666666666667,
 0.03333333333333333,
 0.1,
 0.2,
 0.13333333333333333,
 0.03333333333333333,
 0.0,
 0.0,
 0.6666666666666666,
 0.3333333333333333,
 0.0,
 0.0,
 0.5,
 0.03333333333333333,
 0.06666666666666667,
 0.26666666666666666,
 0.3,
 0.1,
 0.2,
 0.13333333333333333,
 0.5,
 0.2,
 0.23333333333333334,
 0.2,
 0.36666666666666664,
 0.0,
 0.13333333333333333,
 0.13333333333333333,
 0.32142857142857145,
 0.4,
 0.2,
 0.0,
 0.0,
 0.25,
 0.0,
 1.0,
 0.42857142857142855,
 0.25925925925925924,
 0.3157894736842105,
 0.1,
 0.0,
 0.0,
 0.08333333333333333,
 0.3333333333333333,
 0.5333333333333333,
 0.14814814814814814,
 0.2,
 0.1111111111111111,
 0.0,
 0.23333333333333334]

In [25]:
labelledrecord

[0.1,
 0.06666666666666667,
 0.06666666666666667,
 0.1,
 0.3,
 0.16666666666666666,
 0.03333333333333333,
 0.13333333333333333,
 0.0,
 0.6666666666666666,
 0.36666666666666664,
 0.0,
 0.0,
 0.43333333333333335,
 0.2,
 0.2,
 0.43333333333333335,
 0.3,
 0.06666666666666667,
 0.2,
 0.03333333333333333,
 0.8333333333333334,
 0.2,
 0.3333333333333333,
 0.23333333333333334,
 0.5333333333333333,
 0.2,
 0.13333333333333333,
 0.13333333333333333,
 0.17857142857142858,
 0.4,
 0.0,
 0.0,
 0.0,
 0.5,
 0.0,
 1.0,
 0.7142857142857143,
 0.2962962962962963,
 0.3157894736842105,
 0.06666666666666667,
 0.0,
 0.0,
 0.08333333333333333,
 0.0,
 0.6666666666666666,
 0.18518518518518517,
 0.2,
 0.14814814814814814,
 0.0,
 0.23333333333333334]